# Tutorial : Random Coefficients Logit
In this tutorial, we'll use data from :ref:`references:Nevo (2000)` to solve the paper's fake cereal problem. Locations of CSV files that contain the data are in the :mod:`data` module.

In [1]:
%matplotlib inline

import pyblp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pyblp.options.digits = 3
pyblp.options.verbose = False
np.set_printoptions(precision=2, threshold=10, linewidth=100)
pyblp.__version__

'0.5.0'

### Theory of Random Coefficients Logit
The random coefficients model extends the plain logit model by allowing for correlated tastes for different product characteristics $x_{jt}$.
In this  model (indirect) utility is given by:
$$u_{ijt} = x_{jt} \beta_i - \alpha_i p_{jt} + \xi_{jt} + \varepsilon_{ijt}$$
The main addition is that $(\alpha_i,\beta_i)$ have individual specific subscripts $i$.

Conditional on $\beta_i$ the individual marketshares follow the same logit form as before. But now we must integrate over heterogeneous individuals to get the aggregate marketshares:

$$s_{jt}(\alpha,\beta, \theta_2) = \int \frac{e^{x_{jt} \beta_i   - \alpha_i p_{jt}  + \xi_{jt}}}{\sum_k e^{x_{kt} \beta_i   - \alpha_i p_{jt}  + \xi_{kt}}} f(\beta_i,\alpha_i | \theta_2)$$

In general, this integral needs to be performed numerically. This also means that we can't directly linearize the model. It is common to re-parametrize the model to separate the aspects of mean utility that everyone agrees on: 

$$\delta_{jt} = x_{jt} \overline{\beta} - \overline{\alpha} p_{jt} + \xi_{jt}$$
From the individual specific heterogeneity $\mu_{ijt}(\theta_2)$.

\begin{equation}
s_{jt}(\delta_{jt}, \theta_2) = \int \frac{e^{\delta_{jt} + \mu_{ijt}}}{\sum_k e^{\delta_{kt} + \mu_{ikt}}} f(\mu_{it} | \theta_2)
\end{equation}

Given a guess of $\theta_2$ we can solve the system of nonlinear equations for the vector $\delta$ which equates observed and predicted marketshares $s_{jt}(\delta, \theta_2)= \mathscr{s}_{jt}$. Now we can perform a linear IV GMM regression of the form:

$$\delta_{jt}(\theta_2) = x_{jt} \overline{\beta} - \overline{\alpha} p_{jt} + \xi_{jt}$$

The moments are constructed by interacting the predicted residuals $\hat{\xi}_{jt}(\theta_2)$ with instruments $z_{jt}$ to form
$$g(\theta) =\frac{1}{N} \sum_{j,t} \hat{\xi}_{jt}(\theta_2) \times  z_{jt}$$

### Implementation Details
In order to include random coefficients we need to add a _formula_ for the nonlinear parameters $(X_2)$

Just like in the logit case we have the same reserved field names in the __product data__:
- __market_ids__ are the unique market identifiers which we subscript $t$.
- __product_ids__ are the unique product identifiers which we subscript $j$ (these are optional).
- __shares__ specifies the marketshares which need to be between zero and one, and within a __market_id__ $:\sum_{j} s_{jt} \leq 1$.
- __prices__ are prices $p_{jt}$. These have some special properties and are _always_ treated as endogenous.
- __demand_instrumentsX__ are numbered demand instruments. These represent only the _excluded_ instruments. The exogenous regressors in $X_1$ will be added to the set of instruments automatically.

We proceed with the following steps:
1. Load the __product data__ which at a minimum consists of __market_ids__, __shares__, and __prices__ and at least one __demand_instrument__
2. Define the _formula_ for the $X_1$ (linear) demand model.
    - This formula is a `patsy` style formula.
    - It includes a _constant_ by default. To exclude the constant either specify a 0 or a -1.
    - To include fixed effects use the __absorb__ option and specify which variable(s) you would like to absorb
    - `pyblp` does some model reduction (it will exclude the constant if you include FE's and can prevent colinearity if you include multiple FE's).
3. Define a _formula_ for the $X_2$ nonlinear demand model.
    - Include only the variables over which we want random coefficients.
    - Do not absorb or include fixed effects.
    - It will include a random coefficient on the constant (to capture inside good vs. outside good preference) unless you specify not to (with a $0$ or $-1$).
4. Define an _integration method_ to solve the integral in (1) from several available options:
    - Monte Carlo integration (pseudo-random draws)
    - Quadrature with Product Rule
    - Sparse Grids Quadrature (see Heiss and Winchel)
    - See the  :class:`Integration` for more options.
3. Combine the _formulas_ and _product data_ and _integration method_ to construct a class :class:`Problem`.
4. Use the :meth:`Problem.solve` method of the :class:`Problem` to estimate paramters.
    - By default `pyblp` estimates an unrestricted covariance matrix for all random coefficients
    - It is required to specify an initial guess of the _nonlinear parameters_. This serves two purposes: (1) to speed up estimation and (2) any initial guesses of zero indicate to the solver which parameters are restricted to be zero always.

### Specifics of Random Taste Parameters

It is common to assume that $f(\beta_i | \theta_2)$ follows a multivariate normal distribution, and to break it up into three parts:
1. A mean taste which all individuals agree on $\overline{\beta}$: $(K \times 1)$
2. A covariance matrix $\Sigma$: $(K_2 \times K_2)$
3. Interactions with observed demographic data $D_i$: $(D \times 1)$ which we label $\Pi$: $(K_2 \times D)$

Together this gives us that:
$$
\beta_i \sim N(\overline{\beta} + \Pi D_i, \Sigma)
$$

- :meth:`Problem.solve` takes an initial guess of $\Sigma_0$. It guarantees that $\hat{\Sigma}$ (the estimated parameters) have the same sparsity structure as $\Sigma_0$. Thus any zero element of $\Sigma_0$ is restricted to be zero in the solution $\hat{\Sigma}$. For example, a popular restriction is that $\Sigma$ is diagonal, this can be achieved by passing a diagonal matrix as $\Sigma_0$.
- As is common with multivariate normal distributions `pyblp` does not estimate $\Sigma$ directly, rather it estimates it's matrix square root $L$ (Cholesky Root) where $L L' = \Sigma$

### Loading the Fake Cereal Data
The `product_data` argument of :class:`Problem` should be a structured array-like object with fields that store data. Product data can be a structured [NumPy](https://www.numpy.org/) array, a [pandas](https://pandas.pydata.org/) DataFrame, or other similar objects. We'll use NumPy to read the data.

In [2]:
product_data = np.recfromcsv(pyblp.data.NEVO_PRODUCTS_LOCATION, encoding='utf-8')
pd.DataFrame(product_data).head()

,market_ids,city_ids,quarter,product_ids,firm_ids0,firm_ids1,brand_ids,shares,prices,sugar,...,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19
0,C01Q1,1,1,F1B04,1,1,4,0.012417,0.072088,2,...,2.116358,-0.154708,-0.005796,0.014538,0.126244,0.067345,0.068423,0.034800,0.126346,0.035484
1,C01Q1,1,1,F1B06,1,1,6,0.007809,0.114178,18,...,-7.374091,-0.576412,0.012991,0.076143,0.029736,0.087867,0.110501,0.087784,0.049872,0.072579
2,C01Q1,1,1,F1B07,1,1,7,0.012995,0.132391,4,...,2.187872,-0.207346,0.003509,0.091781,0.163773,0.111881,0.108226,0.086439,0.122347,0.101842
3,C01Q1,1,1,F1B09,1,1,9,0.005770,0.130344,3,...,2.704576,0.040748,-0.003724,0.094732,0.135274,0.088090,0.101767,0.101777,0.110741,0.104332
4,C01Q1,1,1,F1B11,1,1,11,0.017934,0.154823,12,...,1.261242,0.034836,-0.000568,0.102451,0.130640,0.084818,0.101075,0.125169,0.133464,0.121111


The product data contains __market_id__, __product_id__, two sets of __firm_id__ (the second are IDs after a simple merger, which are used later), __shares__, __prices__, a number of product characteristics, and some pre-computed excluded __demand_instrumentsX__. The __product_id__'s will be used to construct fixed effects. 

For more information about the instruments and the example data as a whole, refer to the :mod:`data` module.

## Solving the Fake Cereal Problem (without Demographics)

Formulations, product data, and an integration configuration are collectively used to initialize a :class:`Problem`. Once initialized, :meth:`Problem.solve` runs the estimation routine. The arguments to :meth:`Problem.solve` configure how estimation is performed. For example, `optimization` and `iteration` arguments configure the optimization and iteration routines that are used by the outer and inner loops of estimation.

For the fake cereal problem, we'll specify formulations for $X_1$ (linear) and $X_2$ (nonlinear/ random coefficients).
- The formulation for $X_1$ consists of __prices__ and __product_id__ fixed effects, which we will absorb using `absorb` argument of :class:`Formulation`.
- If we were interested in reporting estimates for each fixed effect, we could replace the formulation for $X_1$ with `Formulation('prices + C(product_ids)')`. Absorption of fixed effects yields the same first-stage results as does including them as indicator variables, although results for GMM stages after the first may be slightly different because the two methods can give rise to different weighting matrices.
- Because __sugar__, __mushy__, and the constant are collinear with __product_ids__ we can include them in $X_2$ but not in $X_1$.


We also need to specify an :class:`Integration` option. We consider two alternatives:
1. Monte Carlo draws: We simulate 50 individuals from a random normal distribution.
2. Sparse Grids Integration: We construct sparse-grids to a polynomial accuracy of degree 4.

We estimate three versions of the model:
1. An unrestricted covariance matrix for random tastes using Monte Carlo Integration.
2. An unrestricted covariance matrix for random tastes using Sparse Grids Integration.
3. A restricted diagonal matrix for random tastes using Monte Carlo Integration.

Notice that the only thing that changes when we estimate the restricted covariance is our initial guess of $\Sigma_0$.

In [3]:
# These are X_1 formulas
linear_formula = pyblp.Formulation('0 + prices', absorb='C(product_ids)')
# These are X_2 formulas
nonlinear_formula = pyblp.Formulation('1 + prices + sugar + mushy')

product_formulations = (linear_formula,nonlinear_formula)
product_formulations

(prices + Absorb[C(product_ids)], 1 + prices + sugar + mushy)

In [4]:
# Initialize the integration options
int_mc = pyblp.Integration('monte_carlo', size=50, seed=0)
int_grid = pyblp.Integration('grid', size=4)

problem_mc = pyblp.Problem(product_formulations, product_data, integration=int_mc)
problem_grid = pyblp.Problem(product_formulations, product_data, integration=int_grid)
problem_grid

Dimensions:
 N     T    K1    K2    MD    ED 
----  ---  ----  ----  ----  ----
2256  94    1     4     20    1  

Formulations:
       Column Indices:           0       1       2      3  
-----------------------------  ------  ------  -----  -----
 X1: Linear Characteristics    prices                      
X2: Nonlinear Characteristics    1     prices  sugar  mushy

In [5]:
initial_sigma_res=np.eye(4)
initial_sigma=np.ones((4,4))

# this does the estimation -- it takes a few minutes
results_diag=problem_mc.solve(initial_sigma_res,optimization=pyblp.Optimization('bfgs'))
results_mc=problem_mc.solve(initial_sigma,optimization=pyblp.Optimization('bfgs'))
results_grid=problem_grid.solve(initial_sigma,optimization=pyblp.Optimization('bfgs'))

print("*"*45)
print("Diagonal Covariance: Monte Carlo Integration")
print("*"*45)
print(results_diag)
print("*"*45)
print("Unrestricted Covariance: Monte Carlo Integration")
print("*"*45)
print(results_mc)
print("*"*45)
print("Unrestricted Covariance: Sparse Grids Integration")
print("*"*45)
print(results_grid)

*********************************************
Diagonal Covariance: Monte Carlo Integration
*********************************************
Problem Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:12     2         7            28             7423               24111        +4.04E+05    +3.92E-04  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:       1         prices        sugar        mushy   
------  -----------  -----------  -----------  -----------
  1      +5.25E-02    +0.00E+00    +0.00E+00    +0.00E+00 
        (+1.14E+00)                                       
                                                          
prices                -4.34E-01    +0.00E+00    +

We see that all three models give similar estimates of the price coefficient $\overline{\alpha}\approx -30$. Note that one of the estimated terms on the diagonal of $\Sigma$ is negative. Since the diagonal consists of standard deviations, negative values are unrealistic. When using another optimization routine (like the default SLSQP routine) that supports bounds, these diagonal elements are by default bounded from below by zero.

## Adding Demographics to the Cereal Problem
In order to add demographic data we need to make two changes:
1. We need to load the agent/demographic _dataframe.
2. We need to add an agent formulation to the model.

The __agent data__ has several reserved column names.
- __market_ids__ are the index which link the __agent data__ to the __market_ids__ in the __product data__.
- __weights__ $w_{it}$ is the weight attched to each agent. These should sum to one _for each market_ so that $\sum_{i \in I_t} w_{it} =1$. It is often the case the $w_{it} = \frac{1}{I_t}$ where $I_t$ is the number of agents in market $t$, so that each agent gets equal weight. Other possibilties inculdue quadrature nodes and weights.
- __nodesX__ are the nodes at which the unobserved agent tastes $\nu_{ikt}$ are evaluated. The nodes should be labeled from $[0,\ldots,K_2]$ where $K_2$ is the number of nonlinear parameters (random coefficients).
- Other fields are the realizations of the demographics $d$ themselves.

The __agent formulation__ tells us which columns of demographic information to interact with $X_2$. For example, in the Nevo example:

    pyblp.Formulation('0 + income + age + child')
    
This tells us to include demographic realizations for __income__, __age__, and presence of children __child__, but to ignore other possible demographics when interacting demographics with $X_2$. We should also generally exclude the constant from the demographic formula. To add an additional interaction for __income_squared__ we need to include that in the formula:

    pyblp.Formulation('0 + income + income_squared + age + child')
    


In [12]:
agent_formulation = pyblp.Formulation('0 + income + income_squared + age + child')
agent_data = np.recfromcsv(pyblp.data.NEVO_AGENTS_LOCATION, encoding='utf-8')
pd.DataFrame(agent_data).head()

,market_ids,city_ids,quarter,weights,nodes0,nodes1,nodes2,nodes3,income,income_squared,age,child
0,C01Q1,1,1,0.05,0.434101,-1.500838,-1.151079,0.161017,0.495123,8.331304,-0.230109,-0.230851
1,C01Q1,1,1,0.05,-0.726649,0.133182,-0.500750,0.129732,0.378762,6.121865,-2.532694,0.769149
2,C01Q1,1,1,0.05,-0.623061,-0.138241,0.797441,-0.795549,0.105015,1.030803,-0.006965,-0.230851
3,C01Q1,1,1,0.05,-0.041317,1.257136,-0.683054,0.259044,-1.485481,-25.583605,-0.827946,0.769149
4,C01Q1,1,1,0.05,-0.466691,0.226968,1.044424,0.092019,-0.316597,-6.517009,-0.230109,-0.230851


Now we configure the :class:`Problem` to include the __agent_formulation__ and __agent_data__ which follow the __product_formulations__ and __product_data__. 

Inspecting the attributes of the :class:`Problem` instance helps to confirm that the problem has been configured correctly. For example, inspecting :attr:`Problem.products` and :attr:`Problem.agents` confirms that product data were structured correctly and that agent data were built correctly.

When we display the class, it lists the demographic interactions in the _Formulations_ table. We also report $D=4$, the dimension of the demographic draws in the _Dimensions_ table.

In [13]:
nevo_problem = pyblp.Problem(
    product_formulations,
    product_data,
    agent_formulation,
    agent_data)
nevo_problem

Dimensions:
 N     T    K1    K2    D    MD    ED 
----  ---  ----  ----  ---  ----  ----
2256  94    1     4     4    20    1  

Formulations:
       Column Indices:           0           1           2      3  
-----------------------------  ------  --------------  -----  -----
 X1: Linear Characteristics    prices                              
X2: Nonlinear Characteristics    1         prices      sugar  mushy
       d: Demographics         income  income_squared   age   child

The initialized problem can be solved with :meth:`Problem.solve`. We'll use the same starting values as :ref:`references:Nevo (2000)`. By passing a diagonal matrix of ones as starting values for $\Sigma$, we're choosing to optimize over only variance terms. Similarly, zeros in the starting values for $\Pi$ mean that those parameters will be fixed at zero. To solve the problem, we'll use a non-default unbounded optimization routine that is similar to the default one in Matlab often used to replicate the paper.

We also configure :meth:`Problem.solve` to use 1-step GMM instead of 2-step GMM:

    method = '1s'
We do this to replicate published estimates using the fake cereal example.

In [14]:
initial_sigma = np.diag([0.3302, 2.4526, 0.0163, 0.2441])
initial_pi = [
  [ 5.4819,  0,      0.2037,  0     ],
  [15.8935, -1.2000, 0,       2.6342],
  [-0.2506,  0,      0.0511,  0     ],
  [ 1.2650,  0,     -0.8091,  0     ]
]
results = nevo_problem.solve(
   initial_sigma,
   initial_pi,
   optimization=pyblp.Optimization('bfgs'), method='1s'
)
results

Problem Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:19     1         51           58             34545             108533        +4.56E+00    +6.91E-06  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:       1         prices        sugar        mushy     |   Pi:      income     income_squared      age         child   
------  -----------  -----------  -----------  -----------  |  ------  -----------  --------------  -----------  -----------
  1      +5.58E-01    +0.00E+00    +0.00E+00    +0.00E+00   |    1      +2.29E+00     +0.00E+00      +1.28E+00    +0.00E+00 
        (+1.63E-01)                                         |          (+1.21E+00)                  (+6.31

- Results are similar to those in the original paper with a GMM Objective $4.56$ and a price coeffiient $\alpha = -62.7$. 

- Note that one of the estimated terms on the diagonal of $\Sigma$ is negative. Since the diagonal consists of standard deviations, negative values are difficult to interpret. When using another optimization routine (like the default L-BFGS-B routine) that supports bounds, these diagonal elements are by default bounded from below by zero.



### Restricting Parameters
Because the interactions between __price__ and __income__ and __income_squared__ are potentially collinear we might worry that $\pi_{p,inc} =588$ and  $\pi_{p,inc2} =-30.2$ are pushing the price coefficient in opposite directions and both are large in magnitude but statistically insignficant. One way of dealing with this is to restrict $\pi_{p,inc2} =0$.

There are two ways we can do this:
1. Change the agent formula to drop __income_squared__.
2. Change the initial $\Pi$ values to make this term zero.

In [15]:
# Restrict Initial Guess of Pi so that second column (income_squared) is zero
initial_pi_res = [
  [ 5.4819,  0,      0.2037,  0     ],
  [15.8935,  0,      0,       2.6342],
  [-0.2506,  0,      0.0511,  0     ],
  [ 1.2650,  0,     -0.8091,  0     ]
]

results_res = nevo_problem.solve(
   initial_sigma,
   initial_pi_res,
   optimization=pyblp.Optimization('bfgs')
)

In [16]:
# Now eliminate income_squared from formulation and adjust initial guess to remove the corresponding column
agent_formulation_res = pyblp.Formulation('0 + income  + age + child')
initial_pi_res2 = [
  [ 5.4819, 0.2037,  0     ],
  [15.8935, 0,       2.6342],
  [-0.2506, 0.0511,  0     ],
  [ 1.2650,-0.8091,  0     ]
]

nevo_problem_res = pyblp.Problem(
    product_formulations,
    product_data,
    agent_formulation_res,
    agent_data)

results_res2 = nevo_problem_res.solve(
   initial_sigma,
   initial_pi_res2,
   optimization=pyblp.Optimization('bfgs')
)

In [17]:
print(results_res)
print(results_res2)

Problem Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:31     2         17           47             19755              62916        +4.58E+04    +7.42E-02  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:       1         prices        sugar        mushy     |   Pi:      income     income_squared      age         child   
------  -----------  -----------  -----------  -----------  |  ------  -----------  --------------  -----------  -----------
  1      +3.85E-01    +0.00E+00    +0.00E+00    +0.00E+00   |    1      +3.08E+00     +0.00E+00      +1.38E+00    +0.00E+00 
        (+1.24E-01)                                         |          (+1.10E+00)                  (+1.09

The parameter estimates and standard errors are identical for both approaches. There is some evidence that the sovler took a slightly different path for each problem (based on number of iterations) it appears as if both restricted problems arrived at identical answers. The $\pi_{p,inc}$ interaction term is still insignificant.